<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Конкуретный-анализ" data-toc-modified-id="Конкуретный-анализ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Конкуретный анализ</a></span></li></ul></div>

# Конкуретный анализ

Есть поставщик симок Airalo, которому интересен конкуретный анализ - Airalo.
Цель - спарсить все существующие предложения от поставщика и сопоставить каждому пакету - его цену, и выгрузить все в гугл-таблицу

In [18]:
# Parse
from scrapy import Selector
import requests
import time
import urllib
import bs4 as bs
import itertools
import pandas as pd
import re
# Google Services
from google.oauth2.service_account import Credentials
from google.cloud import bigquery as bq
import gspread
# Auth client
credentials_json = '.\credentials\\big_query_admin.json'
credentials = Credentials.from_service_account_file(credentials_json)
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
creds_with_scope = credentials.with_scopes(scope)
client_gs = gspread.authorize(creds_with_scope)

In [4]:
url='https://www.airalo.com/api/v3/countries?sort=asc'
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'},method='GET')
respons = urllib.request.urlopen(req)
data = json.loads(respons.read().decode(respons.info().get_param('charset') or 'utf-8'))

In [5]:
url_ends = []
for country in data:
    url_ends.append('https://www.airalo.com/'+country['slug']+'-esim')

In [7]:
url_ends[2:5]

['https://www.airalo.com/algeria-esim',
 'https://www.airalo.com/andorra-esim',
 'https://www.airalo.com/anguilla-esim']

In [8]:
def grouper(iterable, n):
    iterable = iter(iterable)
    while True:
        tup = tuple(itertools.islice(iterable, 0, n))
        if tup:
            yield tup
        else:
            break
result = []
column_list = ['coverage','data','validity','price','sale price']
for url in url_ends:
    req = urllib.request.Request(url,
                                 headers={'User-Agent': 'Mozilla/5.0'},
                                 method='GET')
    #
    html = urllib.request.urlopen(req).read()
    #
    data = bs.BeautifulSoup(html).findAll('p',{'data-testid':'promotional-price-value'})
    values = [tag.get_text().strip() for tag in data]
    units = list(grouper(values,4))
    for i in range(len(units)):
        row = {column: unit for column,unit in zip(column_list,units[i])}
        result.append(row)

In [11]:
df = pd.DataFrame(result)
df.head(10)

,coverage,data,validity,price
0,Afghanistan,1 GB,7 Days,US $6.50
1,Afghanistan,2 GB,15 Days,US $12.00
2,Afghanistan,3 GB,30 Days,US $17.00
3,Afghanistan,5 GB,30 Days,US $25.50
4,Albania,1 GB,7 Days,US $4.50
5,Albania,2 GB,15 Days,US $8.00
6,Albania,3 GB,30 Days,US $11.00
7,Albania,5 GB,30 Days,US $15.50
8,Albania,10 GB,30 Days,US $26.00
9,Algeria,1 GB,7 Days,US $7.00


In [12]:
def price_cleaner(x):
    l = re.search(r'(?:US\s)?\$(\d+\.\d{1,2})',str(x))
    if l:
        return l.group(1)
    else:
        return np.nan   
df.price = df.price.apply(price_cleaner)
df = df[df.price.notna()].copy()
df.price = df.price.astype(float)

In [13]:
df = df.query('data in ("1 GB","2 GB","3 GB","5 GB")')\
    .pivot_table(index='coverage',columns='data',aggfunc={'price':'sum'})

In [14]:
df.columns = ["Airalo 1 GB, $","Airalo 2 GB, $","Airalo 3 GB, $","Airalo 5 GB, $"]
df.index.name = 'Country'
df

,"Airalo 1 GB, $","Airalo 2 GB, $","Airalo 3 GB, $","Airalo 5 GB, $"
Country,,,,
Afghanistan,6.5,12.0,17.0,25.5
Albania,4.5,8.0,11.0,15.5
Algeria,7.0,13.0,17.5,27.0
Andorra,4.5,8.0,10.5,15.0
Anguilla,9.5,NaN,NaN,NaN
...,...,...,...,...
Vatican City,4.5,8.0,10.0,14.5
Venezuela,9.5,18.0,25.5,40.0
Vietnam,4.5,7.0,9.5,13.0


In [20]:
def upload_data_gsp(url,df):
    '''
    url - url-link for google sheet (Ссылка на таблицу)
    df - frame for write
    '''
    #1. Get SpreadSheet
    spreadsheet = client_gs.open_by_url(url)
    #2. Get data from one Sheet with interval (first row - header)
    client_gs.import_csv(spreadsheet.id, df.to_csv())
    return None

In [21]:
upload_data_gsp('https://docs.google.com/spreadsheets/d/1xZZ-lbSprhRu_LDp1P4uK_cTL_gqdzjTuMLYiq2TJGc/edit?usp=sharing',df)

Ссылка на гугл-таблицу с результатом [парсинга](https://docs.google.com/spreadsheets/d/1xZZ-lbSprhRu_LDp1P4uK_cTL_gqdzjTuMLYiq2TJGc/edit#gid=7513511)